In [ ]:
import pymol2
with pymol2.PyMOL() as pymol:
    pymol.cmd.load('wTTC7A.SwMod.relax.attempt3.pdb', 'model')
    interfaceA3 = [int(atom.resi) for atom in pymol.cmd.get_model('(byres chain A around 3) and chain B and name CA').atom]
    interfaceA5 = [int(atom.resi) for atom in pymol.cmd.get_model('(byres chain A around 5) and chain B and name CA').atom]
    pymol.cmd.delete('*')
    pymol.cmd.load('6bq1.relax.pdb', 'model')
    interfaceB3 = [int(atom.resi) for atom in pymol.cmd.get_model('(byres chain A around 3) and chain B and name CA').atom]
    interfaceB5 = [int(atom.resi) for atom in pymol.cmd.get_model('(byres chain A around 5) and chain B and name CA').atom]

In [ ]:
from docx import Document
from docx.shared import Pt, RGBColor
from docx.enum.text import WD_COLOR_INDEX
import re

document = Document()

document.add_heading('Alignment of TTC7A-B', 0)
p = document.add_paragraph()

def chunkify(sequence):
    n = 10
    if isinstance(sequence, Seq):
        seq = str(sequence)
    else:
        seq = sequence
    return [seq[i:i+n] for i in range(0, len(seq), n)]

def monospace(paragraph, text):
    run = paragraph.add_run(text)
    run.font.name = 'Andale Mono'
    run.font.size = Pt(10)
    return run

def monospace_residues(paragraph, block, colors3, colors5):
    for r, c3, c5 in zip(block, colors3, colors5):
        for rr, cc3, cc5 in zip(r, c3, c5):
            run = monospace(paragraph, rr)
            if cc3:
                #run.font.color.rgb = RGBColor(0x00, 0x00, 0xff) #1B4F72 
                run.font.highlight_color = WD_COLOR_INDEX.BLUE
                run.font.color.rgb = RGBColor(0xff, 0xff, 0xff)
            elif cc5:
                run.font.highlight_color = WD_COLOR_INDEX.TURQUOISE
        monospace(paragraph, ' ')
    monospace(paragraph, '\n')

def flip(sequence, interface):
    imap = []
    i = 1
    for r in str(sequence):
        if r == '-':
            imap.append(False)
        elif i in interface:
            imap.append(True)
            i += 1
        else:
            imap.append(False)
            i += 1
    return imap

imapA3 = flip(sequences['TTC7A_HUMAN'], interfaceA3)
imapB3 = flip(sequences['TTC7B_HUMAN'], interfaceB3)
imapA5 = flip(sequences['TTC7A_HUMAN'], interfaceA5)
imapB5 = flip(sequences['TTC7B_HUMAN'], interfaceB5)

A, B = chunkify(sequences['TTC7A_HUMAN']), chunkify(sequences['TTC7B_HUMAN'])
iA3, iB3 = chunkify(imapA3), chunkify(imapB3)
iA5, iB5 = chunkify(imapA5), chunkify(imapB5)
for i in range(0, len(A), 5):
    r= monospace(p, f'TTC7A_HUMAN ')
    r.bold = True
    monospace_residues(p, A[i:i+5], iA3[i:i+5], iA5[i:i+5])
    r= monospace(p, f'TTC7B_HUMAN ')
    r.bold = True
    monospace_residues(p, B[i:i+5], iB3[i:i+5], iB5[i:i+5])
    monospace(p, '\n')
    
document.save('test.docx')

## Get interface

In [ ]:
import pymol2

resis = {}
chain = 'A'
nglise = lambda arr: ' or '.join([f'{v}:{chain}' for v in arr])

for filename in ('wTTC7A.SwMod.relax.attempt3.pdb', '6bq1.relax.pdb'):
    with pymol2.PyMOL() as pymol:
        pymol.cmd.load(filename)
        at = pymol.cmd.get_model('chain A and name CA and byres (chain B around 4)').atom
        inter = sorted([a.resi for a in at])
        resis[filename] = inter
        print(filename, len(inter))
        print(nglise(inter))
        print('+'.join(inter))
        print(inter)

In [ ]:
a = set(resis['wTTC7A.SwMod.relax.attempt3.pdb'])
b = set(resis['6bq1.relax.pdb'])

print('color tv_green, chain A and resi '+'+'.join(b & a))
print('color tv_blue, chain A and resi '+'+'.join(b - a))
print('color tv_red, chain A and resi '+'+'.join(a - b))

print('A', nglise(a))
print('B', nglise(b))
print('COMMON', nglise(b & a))
print('UNIQUE-B', nglise(b - a))
print('UNIQUE-A', nglise(a - b))